In [21]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [3]:
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [23]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [24]:
vote_file = pd.read_csv('brexit_full_updated3.csv')
vote_file

,Constituency ID,full name,PANO,Constituency Name,Party abbreviation,conShare,conShare2,conShare3,bill161,leave,party_old,ref,ref_dummy,party,old_party
0,E14000530,Gerald Howarth,7,Aldershot,Con,50.59,73.403947,73.403947,1.0,1.0,1.0,0.578978,1,1.0,1.0
1,E14000531,Wendy Morton,8,Aldridge-Brownhills,Con,52.05,69.940876,69.940876,1.0,0.0,1.0,0.677963,1,0.0,0.0
2,E14000532,Graham Brady,9,Altrincham and Sale West,Con,52.99,66.503514,66.503514,1.0,1.0,1.0,0.385878,0,1.0,1.0
3,E14000533,Nigel Mills,11,Amber Valley,Con,43.98,55.840528,55.840528,1.0,1.0,1.0,0.652991,1,1.0,1.0
4,E14000534,Nick Herbert,18,Arundel and South Downs,Con,60.79,84.442284,84.442284,NaN,0.0,1.0,0.497011,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,W07000076,Wayne David,114,Caerphilly,Lab,16.59,27.223499,27.223499,1.0,0.0,0.0,0.551360,1,0.0,NaN
656,W07000077,Chris Evans,336,Islwyn,Lab,15.16,23.639482,23.639482,1.0,0.0,0.0,0.589399,1,0.0,NaN
657,W07000078,Alun Cairns,589,Vale Of Glamorgan,Con,46.02,58.527280,58.527280,1.0,0.0,1.0,0.525507,1,0.0,0.0
658,W07000079,Kevin Brennan,129,Cardiff West,Lab,25.15,38.221884,38.221884,0.0,0.0,0.0,0.438226,0,0.0,NaN


In [25]:
def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return t
        
    return topic

In [26]:
politicians_party = {}
politicians_ref = {}
politicians = vote_file['full name'].unique()
for index, row in vote_file.iterrows():
    politicians_party[row['full name']] = row['party']
    politicians_ref[row['full name']] = row['ref_dummy']

In [57]:
df1 = pd.read_csv('2015_commons.csv')
df2 = pd.read_csv('2016_commons.csv')
df = pd.concat([df1, df2])
df = df.drop(['Government'], axis=1)
df = df.loc[(df['Party'] == 'Conservative') | (df['Party'] == 'Labour')]
df = df.loc[df['Name'].isin(politicians)]
df['Date'] = pd.to_datetime(df['Date'])
mask = (df['Date'] > '2015-6-1') & (df['Date'] <= '2016-6-30')
df = df.loc[mask]
df['party2'] = df['Name'].apply(lambda x: politicians_party[x])
df['ref_dummy'] = df['Name'].apply(lambda x: politicians_ref[x])
df['Topic'] = df['Topic'].apply(lambda x: preprocess(x))
# df = df.loc[df['Topic'] == 'Parliament, government and politics']
# df = df.loc[df['Topic'] == 'European Union']
# df = df.loc[df['Topic'] == 'Crime, civil law, justice and rights']
df = df.loc[df['Topic'] == 'Health services and medicine']
# df = df.loc[df['Topic'] == 'International affairs']
cols = df.columns
cols = ['date', 'speaker', 'name', 'Party', 'Topic', 'transcript', 'party2', 'ref_dummy']
df.columns = cols
df.head()

,date,speaker,name,Party,Topic,transcript,party2,ref_dummy
19224,2015-06-02,Mrs Emma Lewell-Buck (South Shields) (Lab),Emma Lewell-Buck,Labour,Health services and medicine,1. What assessment he has made of recent tren...,0.0,1
19225,2015-06-02,The Secretary of State for Health (Mr Jeremy H...,Jeremy Hunt,Conservative,Health services and medicine,"As you said, Mr Speaker, we shall have those t...",0.0,0
19226,2015-06-02,Mrs Lewell-Buck,Emma Lewell-Buck,Labour,Health services and medicine,I echo the comments made about the late Member...,0.0,1
19227,2015-06-02,Mr Hunt,Jeremy Hunt,Conservative,Health services and medicine,I take responsibility for everything that happ...,0.0,0
19228,2015-06-02,Mr James Gray (North Wiltshire) (Con),James Gray,Conservative,Health services and medicine,I stood against Charles Kennedy in 1992 in Ros...,1.0,1


In [58]:
X = []

for index, row in df.iterrows():
    party = row['Party']
    if party == 'Conservative':
        X.append([row['transcript'], 0])
    if party == 'Labour':
        X.append([row['transcript'], 1])
        
train_df = pd.DataFrame(X, columns=['text', 'labels'])

In [59]:
model_args = ClassificationArgs(sliding_window=True, overwrite_output_dir=True)

model = ClassificationModel(
    "distilbert", "distilbert-base-uncased", use_cuda=False, args=model_args
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [60]:
model

In [61]:
model.train_model(train_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/4024 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1397 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1048 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (692 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1226 [00:00<?, ?it/s]

/home/ubuntu/anaconda3/envs/econ/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.


(1226, 0.4709489710483135)

In [1]:
test_df = pd.read_csv('test_key_speeches.csv')
X_test = list(test_df['text'].values)
        

NameError: name 'pd' is not defined

In [63]:
predictions, raw_outputs = model.predict(X_test)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1089 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1086 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1162 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1042 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 1089 features created from 1089 samples.


  0%|          | 0/153 [00:00<?, ?it/s]

In [64]:
raw_outputs[0]

array([[ 0.6569438, -0.680619 ]])

In [2]:
outputs = [output[0] for output in raw_outputs]
outputs

NameError: name 'raw_outputs' is not defined

In [66]:
from scipy.special import softmax
outputs = softmax(outputs, axis=1)

In [67]:
import numpy as np

In [68]:
rows = []
class_map ={
    0: 'conservative',
    1: 'labour'
}
print(len(X_test))
print(len(outputs))
assert(len(X_test) == len(outputs))
for i in range(len(outputs)):
    output = outputs[i]
    row = [X_test[i], output[0], output[1], class_map[np.argmax(output)]]
    rows.append(row)

res_df = pd.DataFrame(rows, columns=['text', 'conservative prob', 'labour prob', 'label'])
res_df

1089
1089


,text,conservative prob,labour prob,label
0,We all agree that it is vital that everyone is...,0.792089,0.207911,conservative
1,The IN campaign maintains that there is no pro...,0.123088,0.876912,labour
2,It is Government policy that Turkey should joi...,0.163760,0.836240,labour
3,It is also the policy of the European Union th...,0.857251,0.142749,conservative
4,It is also a fact that both the European Union...,0.904083,0.095917,conservative
...,...,...,...,...
1084,This chance may never come again in our lifeti...,0.870314,0.129686,conservative
1085,BAGEHOT: The prime minister is nearing the end...,0.059040,0.940960,labour
1086,BAGEHOT: He’s asking for things he knows he’s ...,0.057449,0.942551,labour
1087,BAGEHOT: You mention the Euro campaign. Are th...,0.319345,0.680655,labour


In [69]:
res_df['label'].value_counts()

conservative    726
labour          363
Name: label, dtype: int64

In [70]:
res_df.to_csv('key_speeches_labour_conservative_prediction.csv', index=False)

In [71]:
train_df

,text,labels
0,1. What assessment he has made of recent tren...,1
1,"As you said, Mr Speaker, we shall have those t...",0
2,I echo the comments made about the late Member...,1
3,I take responsibility for everything that happ...,0
4,I stood against Charles Kennedy in 1992 in Ros...,0
...,...,...
4019,What the hon. Gentleman is saying very much ec...,1
4020,"There is no pressure here, then. I thank my fr...",0
4021,I take the hon. Gentleman’s point. I cannot be...,0
4022,It is a pleasure and an honour to have secured...,0


In [56]:
train_df['labels'].value_counts()

0    3369
1    1566
Name: labels, dtype: int64

In [ ]:
model.eval_model(train_df)